In [0]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minzzzzi","key":"74a980fb52033cf0ead89c84120a4a9c"}'}

In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.4MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=fdb7c7ca0b448ec588d3d0b5afe2cf8fb92cb7e844be1aa901f8b1114d93ff78
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Apr 27 07:16 kaggle.json
-rw-r--r-- 1 root root 64 Apr 27 07:16 kaggle.json


In [0]:
! kaggle competitions download -c classification-animal-classification

  0% 0.00/999 [00:00<?, ?B/s]
100% 999/999 [00:00<00:00, 1.05MB/s]


In [0]:
! unzip classification-animal-classification.zip

Archive:  classification-animal-classification.zip
  inflating: test_des.csv            
  inflating: test_summit_form.csv    
  inflating: train.csv               


In [0]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

torch.manual_seed(1)

In [0]:
xy = pd.read_csv('train.csv', header= None)

x_data = xy.loc[:,0:15]
y_data = xy[16]
x_data = np.array(x_data)
y_data = np.array(y_data)

x_train = torch.FloatTensor(x_data)
y_train = torch.LongTensor(y_data)

In [0]:
print(x_train[:5])
print(y_train[:5])
print(x_train.shape)


tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])
tensor([0, 0, 3, 0, 0])
torch.Size([81, 16])


In [0]:
import torch.nn.functional as F

nb_class = 7
nb_data = len(y_train)

W = torch.zeros((16,7),requires_grad= True)
b = torch.zeros(1,requires_grad= True)

optimizer = optim.SGD([W, b], lr = 0.02)
nb_epochs = 100000

for epoch in range(nb_epochs + 1):

  hypo = F.softmax(x_train.matmul(W)+b,dim = 1)

  #y_one_hot = torch.zeros(nb_data,nb_class)
  #y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1) , 1)
  #cost = (y_one_hot * -torch.log(F.softmax(hypo,dim = 1))).sum(dim = 1).mean()
  cost = F.cross_entropy((x_train.matmul(W) + b), y_train)
  

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 10000 == 0:
    print('Epoch {:4d}/{} Cost {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/100000 Cost 1.945909
Epoch 10000/100000 Cost 0.030607
Epoch 20000/100000 Cost 0.015150
Epoch 30000/100000 Cost 0.010054
Epoch 40000/100000 Cost 0.007527
Epoch 50000/100000 Cost 0.006018
Epoch 60000/100000 Cost 0.005016
Epoch 70000/100000 Cost 0.004302
Epoch 80000/100000 Cost 0.003767
Epoch 90000/100000 Cost 0.003351
Epoch 100000/100000 Cost 0.003018


In [0]:
hypo = F.softmax(x_train.matmul(W)+b, dim = 1)
predict = torch.argmax(hypo, dim = 1)

print(predict[:5])

correct_prediction = predict.float() == y_train

print(correct_prediction[:5])
accuracy = correct_prediction.sum().item()/len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([0, 0, 3, 0, 0])
tensor([True, True, True, True, True])
The model has an accuracy of 100.00% for the training set.


In [0]:
test = pd.read_csv('test_des.csv',header=None)
x_data = test.loc[:,:15]

x_data = np.array(x_data)
x_test = torch.FloatTensor(x_data)

hypo = F.softmax(x_test.matmul(W) + b, dim = 1)
predict = torch.argmax(hypo, dim = 1)
print(x_data.shape)

(20, 16)


In [0]:
predict

tensor([5, 3, 1, 0, 6, 3, 1, 5, 4, 1, 4, 3, 0, 0, 1, 0, 5, 0, 5, 1])

In [0]:
predict = predict.numpy().reshape(-1,1)
id = np.array([i for i in range(len(x_data))]).reshape(-1,1)
result = np.hstack([id,predict])
df = pd.DataFrame(result,columns = ["Id", "Category"])


In [0]:
df.to_csv("동물분류_1801109_엄민지.csv",index = False, header= True)

In [0]:
!kaggle competitions submit -c classification-animal-classification -f 동물분류_1801109_엄민지.csv -m "18011809 엄민지"

100% 102/102 [00:00<00:00, 394B/s]
Successfully submitted to Classification : Animal Classification